![alt text](Backtesting.png)
Image credit: Investopedia

### 🎯 用台積電(2330)過去一年的歷史數據來回測黃金交叉和死亡交叉這個交易策略。

當短期移動平均線上穿長期移動平均線（黃金交叉）出現時，買入資產，並持有直到出現短期移動平均線下穿長期移動平均線（死亡交叉）。

免責聲明

此影片以教學為目的而製作，其內容不構成投資建議。再者，回測的數據不能保證未來實際交易的結果。

#### 回測主要包含四個部分：
1. 取得歷史數據
2. 整理數據
3. 執行策略邏輯
4. 評估測試結果

### <span style="color: #89bafe;">1. 取得歷史數據</span>

In [72]:
# import the necessary packages
import yfinance as yf
import pandas as pd

In [73]:
# Get the data for the stock TW.2330
data = yf.download('2330.TW', start='2023-07-23', end='2024-07-24')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-07-24,557.0,563.0,557.0,558.0,547.339417,25268835
2023-07-25,561.0,569.0,561.0,569.0,558.129272,19590349
2023-07-26,569.0,571.0,563.0,566.0,555.186646,13314305
2023-07-27,570.0,570.0,566.0,569.0,558.129272,12200358
2023-07-28,569.0,573.0,565.0,567.0,556.167480,18540078
...,...,...,...,...,...,...
2024-07-17,1035.0,1045.0,1020.0,1030.0,1030.000000,54022399
2024-07-18,988.0,1005.0,986.0,1005.0,1005.000000,81990095
2024-07-19,988.0,995.0,970.0,970.0,970.000000,92507621


### <span style="color: #89bafe;">2. 整理數據</span>

In [74]:
# add columns: sma_5, sma_20
data['sma_5'] = data['Close'].rolling(window=5).mean()
data['sma_20'] = data['Close'].rolling(window=20).mean()

# # add columns: prev_sma_5, prev_sma_20
data['prev_sma_5'] = data['sma_5'].shift(1)
data['prev_sma_20'] = data['sma_20'].shift(1)

# # add columns: next_open
data['next_open'] = data['Open'].shift(-1)

# # drop rows with NaN
data.dropna(inplace=True)

data

,Open,High,Low,Close,Adj Close,Volume,sma_5,sma_20,prev_sma_5,prev_sma_20,next_open
Date,,,,,,,,,,,
2023-08-22,544.0,544.0,538.0,541.0,530.664185,14594304,540.6,553.25,540.8,554.10,546.0
2023-08-23,546.0,552.0,544.0,552.0,541.454102,20636332,542.6,552.40,540.6,553.25,563.0
2023-08-24,563.0,564.0,558.0,564.0,553.224792,23280827,546.6,552.30,542.6,552.40,549.0
2023-08-25,549.0,551.0,546.0,546.0,535.568726,24144537,548.0,551.15,546.6,552.30,547.0
2023-08-28,547.0,553.0,547.0,549.0,538.511414,8991985,550.4,550.25,548.0,551.15,551.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-07-16,1040.0,1070.0,1035.0,1055.0,1055.000000,32125609,1052.0,997.75,1049.0,992.15,1035.0
2024-07-17,1035.0,1045.0,1020.0,1030.0,1030.000000,54022399,1049.0,1000.20,1052.0,997.75,988.0
2024-07-18,988.0,1005.0,986.0,1005.0,1005.000000,81990095,1034.0,1001.40,1049.0,1000.20,988.0


### <span style="color: #89bafe;">3. 執行策略邏輯</span>

In [75]:
has_stock = False
buy_list = []
sell_list = []

for index, row in data.iterrows():
    if not has_stock:
        # golden cross
        if row['sma_5'] > row['sma_20'] and row['prev_sma_5'] <= row['prev_sma_20']:
            has_stock = True
            # append next index and next_open
            next_index = index + pd.Timedelta(days=1)
            buy_list.append((next_index, row['next_open']))
    else:
        # death cross
        if row['sma_5'] < row['sma_20'] and row['prev_sma_5'] >= row['prev_sma_20']:
            has_stock = False
            # append next index and next_open
            next_index = index + pd.Timedelta(days=1)
            sell_list.append((next_index, row['next_open']))

if len(buy_list) != len(sell_list):
    # remove the last item in the buy_list
    buy_list.pop()

# buy_list & sell_list to DataFrame
buy_df = pd.DataFrame(buy_list, columns=['date_buy', 'price_buy'])
sell_df = pd.DataFrame(sell_list, columns=['date_sell', 'price_sell'])
# merge buy_df and sell_df
df = pd.concat([buy_df, sell_df], axis=1)

# add column: return_rate
df['return_rate'] = round((df['price_sell'] - df['price_buy']) / df['price_buy'], 3)
# 
df



,date_buy,price_buy,date_sell,price_sell,return_rate
0,2023-08-29,551.0,2023-09-12,536.0,-0.027
1,2023-10-13,550.0,2023-10-31,535.0,-0.027
2,2023-11-08,550.0,2023-12-08,574.0,0.044
3,2023-12-15,585.0,2024-01-09,588.0,0.005
4,2024-01-16,581.0,2024-01-17,583.0,0.003
5,2024-01-20,633.0,2024-04-23,761.0,0.202
6,2024-05-09,798.0,2024-06-06,893.0,0.119
7,2024-06-07,885.0,2024-07-23,963.0,0.088


### <span style="color: #89bafe;">4. 評估測試結果</span>

In [76]:
# win rate
win_rate = round(len(df[df['return_rate'] > 0]) / len(df), 3)
win_rate

0.75

In [77]:
# compare the return rate of the strategy with the buy-and-hold return rate
strategy_return_rate = round(df['return_rate'].sum(),3)
strategy_return_rate

0.407

In [78]:
# calculate the buy-and-hold return rate
buy_and_hold_return_rate = round((data['Open'].iloc[-1] - data['Open'].iloc[0]) / data['Open'].iloc[0], 3)
buy_and_hold_return_rate

0.772